This is a basic book recommendation AI that suggests similar books based on a title you input.
The dataset used here is small, so it does not include a wide variety of books and might return limited recommendations.
Steps to use:  

1) Run Cell 1 and upload the data.zip file provided.
2) Run Cell 2 and type a book name

(Examples: The Da Vinci Code, Wild Animus, The Firm, A Time to Kill)

In [13]:

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from google.colab import files
import zipfile

print("Please upload your data.zip file")
uploaded = files.upload()
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall()

books_df = pd.read_csv("Books.csv", low_memory=False)
ratings_df = pd.read_csv("Ratings.csv", low_memory=False)

book_counts = ratings_df['ISBN'].value_counts()
valid_books = book_counts[book_counts >= 30].index
ratings_df = ratings_df[ratings_df['ISBN'].isin(valid_books)]

book_user_matrix = ratings_df.pivot_table(
    index='ISBN', columns='User-ID', values='Book-Rating'
).fillna(0)

book_user_sparse = csr_matrix(book_user_matrix.values)
similarity_matrix = cosine_similarity(book_user_sparse)
isbn_list = book_user_matrix.index.tolist()
isbn_to_title = books_df.set_index('ISBN')['Book-Title'].to_dict()



Please upload your data.zip file


Saving data.zip to data (2).zip


In [15]:
def recommend_books(book_title, top_n=5):
    matching_books = books_df[books_df['Book-Title'].str.lower() == book_title.lower()]

    if matching_books.empty:
        return f"Sorry, no book found with the title '{book_title}'. Please check the spelling."

    target_isbn = matching_books.iloc[0]['ISBN']

    if target_isbn not in isbn_list:
        return f"We found the book '{book_title}', but it does not have enough rating data to generate recommendations"

    idx = isbn_list.index(target_isbn)
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    recommended_books = []
    for i, score in similarity_scores:
        isbn = isbn_list[i]
        title = isbn_to_title.get(isbn, "Unknown Title")
        recommended_books.append((title, round(float(score), 3)))  # Clean up float formatting

    return recommended_books


# user input
book_title = input("Enter a book title you like: ")

results = recommend_books(book_title)

if isinstance(results, str):
    print(results)
else:
    print(f"\n If you liked {book_title}, you might also enjoy:\n")
    for i, (title, score) in enumerate(results, 1):
        print(f"{i}. {title} ")


Enter a book title you like: A Time to Kill

 If you liked A Time to Kill, you might also enjoy:

1. The Pelican Brief 
2. The Client 
3. The Firm 
4. The Chamber 
5. Silence of the Lambs 
